<a href="https://colab.research.google.com/gist/up1512001/1657aac9693accd561be484a1a7c0d3a/video-intelegence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# !zip "/content/drive/MyDrive/6th sem project/Avenue_Dataset.zip" "/content/project code"

In [17]:
from keras.preprocessing.image import img_to_array,load_img
import numpy as np
import glob
import os 
import cv2

from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
import imutils

In [18]:
store_image=[]
train_path='/content/drive/MyDrive/Colab Notebooks/train_video_6th_project'
fps=5
train_videos=os.listdir(train_path)
# print(train_videos)
train_images_path=train_path+'/frames'
os.makedirs(train_images_path)
# print(os.listdir(train_images_path))
def store_inarray(image_path):
    image=load_img(image_path)
    image=img_to_array(image)
    image=cv2.resize(image, (227,227), interpolation = cv2.INTER_AREA)
    gray=0.2989*image[:,:,0]+0.5870*image[:,:,1]+0.1140*image[:,:,2]
    store_image.append(gray)

In [19]:
# from logging import currentframe
currentframe=0
for video in train_videos:
    cap = cv2.VideoCapture('{}/{}'.format(train_path,video))
    while(True):
      ret,frame = cap.read()
      if ret:
        x = '/content/train dir/'+str(currentframe)+'.jpg'
        # print(x)
        cv2.imwrite(x,frame)
        currentframe+=1
      else:
        break
    # os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(train_path,video,fps,train_path))
    # print('ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(train_path,video,fps,train_path))
    # os.system('{}/{} ')
    # print(video)


In [20]:
train_images_path = '/content/train dir'
images=os.listdir(train_images_path)
# images.remove('.ipynb_checkpoints')
  # print(images)
for image in images:
    image_path=train_images_path + '/' + image
    store_inarray(image_path)
    # print(image.shape)

In [21]:
# print(os.listdir('/content/train dir'))

In [22]:
store_image=np.array(store_image)
a,b,c=store_image.shape
# print(store_image.shape)
store_image.resize(b,c,a)
store_image=(store_image-store_image.mean())/(store_image.std())
store_image=np.clip(store_image,0,1)
np.save('training.npy',store_image)

In [23]:
stae_model=Sequential()

stae_model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
stae_model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))
stae_model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

stae_model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [25]:
import tensorflow as tf

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

training_data=np.load('training.npy')
frames=training_data.shape[2]
frames=frames-frames%10

training_data=training_data[:,:,:frames]
training_data=training_data.reshape(-1,227,227,10)
training_data=np.expand_dims(training_data,axis=4)
target_data=training_data.copy()

epochs=5
batch_size=1

callback_save = ModelCheckpoint("saved_model.h5", monitor="mean_squared_error", save_best_only=True)

callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3)

stae_model.fit(training_data,target_data, batch_size=batch_size, epochs=epochs, callbacks = [callback_save,callback_early_stopping])
stae_model.save("saved_model.h5")

Epoch 1/5
1532/1532 [==============================] - 6123s 4s/step - loss: 0.0838 - accuracy: 0.7183
Epoch 2/5
1532/1532 [==============================] - 7186s 5s/step - loss: 0.0690 - accuracy: 0.7374
Epoch 3/5
1532/1532 [==============================] - 7417s 5s/step - loss: 0.0683 - accuracy: 0.7380
Epoch 4/5
1532/1532 [==============================] - 7334s 5s/step - loss: 0.0385 - accuracy: 0.7650
Epoch 5/5
1532/1532 [==============================] - 7315s 5s/step - loss: 0.0287 - accuracy: 0.7758


In [26]:
def mean_squared_loss(x1,x2):
    difference=x1-x2
    a,b,c,d,e=difference.shape
    n_samples=a*b*c*d*e
    sq_difference=difference**2
    Sum=sq_difference.sum()
    distance=np.sqrt(Sum)
    mean_distance=distance/n_samples
    return mean_distance
    

In [39]:
model=tf.keras.models.load_model("saved_model.h5")
cap = cv2.VideoCapture("/content/drive/MyDrive/Colab Notebooks/train_video_6th_project/test/09.avi")
print(cap.isOpened())

True


In [31]:
from google.colab.patches import cv2_imshow

In [1]:
while cap.isOpened():
    imagedump=[]
    ret,frame=cap.read()
    for i in range(10):
        ret,frame=cap.read()
        image = imutils.resize(frame,width=700,height=600)
        frame=cv2.resize(frame, (227,227), interpolation = cv2.INTER_AREA)
        gray=0.2989*frame[:,:,0]+0.5870*frame[:,:,1]+0.1140*frame[:,:,2]
        gray=(gray-gray.mean())/gray.std()
        gray=np.clip(gray,0,1)
        imagedump.append(gray)
    imagedump=np.array(imagedump)
    imagedump.resize(227,227,10)
    imagedump=np.expand_dims(imagedump,axis=0)
    imagedump=np.expand_dims(imagedump,axis=4)
    output=model.predict(imagedump)
    loss=mean_squared_loss(imagedump,output)
    if frame.any()==None:
        print("none")
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
    if loss>0.00068:
        print('Abnormal Event Detected')
        cv2.putText(image,"Abnormal Event",(100,80),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)
    cv2_imshow(image)
cap.release()
cv2.destroyAllWindows()

NameError: ignored

In [41]:
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [42]:
!ls saved_model/my_model

assets	keras_metadata.pb  saved_model.pb  variables


In [49]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
# model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
# loss, acc = model.evaluate(, test_labels, verbose=2)
# print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

In [2]:
tf --version

NameError: ignored

In [3]:
import tensorflow as tf
print(tf)

<module 'tensorflow' from '/usr/local/lib/python3.7/dist-packages/tensorflow/__init__.py'>
